# Higher-order functions

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import _root_.org.scalatest._

Downloaded https://repo1.maven.org/maven2/org/scalatest/scalatest_2.12/3.0.8/scalatest_2.12-3.0.8.pom
Downloaded https://repo1.maven.org/maven2/org/scalactic/scalactic_2.12/3.0.8/scalactic_2.12-3.0.8.pom
Downloaded https://repo1.maven.org/maven2/org/scalactic/scalactic_2.12/3.0.8/scalactic_2.12-3.0.8-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scalactic/scalactic_2.12/3.0.8/scalactic_2.12-3.0.8.jar
Downloaded https://repo1.maven.org/maven2/org/scalatest/scalatest_2.12/3.0.8/scalatest_2.12-3.0.8-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scalatest/scalatest_2.12/3.0.8/scalatest_2.12-3.0.8.jar


import $ivy.$                               

import _root_.org.scalatest._

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

In [ ]:
def compose[A,B,C](f: B => C, g: A => B): A => C =
    (a: A)=> f(g(a))

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [2]:
def sum(list: List[Int]): Int = 
    list match{
        case Nil => 0
        case head::tail=> head + sum(tail)
    }

defined function sum

In [3]:
def multiply(list: List[Int]): Int = 
     list match{
        case Nil => 1
        case head::tail=> head * multiply(tail)
    }

defined function multiply

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [4]:
def foldRight(list: List[Int])(
    nil:Int,
    cons:(Int,Int)=>Int): Int = 
    list match{
        case Nil => nil
        case head::tail=> 
         val tailSol:Int = foldRight(tail)(nil,cons)
         cons(head,tailSol):Int
    }

defined function foldRight

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [5]:
def sum(list: List[Int]): Int = 
    foldRight(list)(
        0,
        (head: Int, tailSum: Int)=> head + tailSum : Int)

defined function sum

In [6]:
def sum(list: List[Int]): Int = 
    foldRight(list)(0,_+_)

defined function sum

In [9]:
sum(List(1,2,3,4,5))

res8: Int = 15

In [7]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1,
        (head: Int, tailMul: Int)=> head * tailMul : Int)

defined function multiply

In [8]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(1,_*_)

defined function multiply

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

In [10]:
def foo(list:List[Int]): Boolean =
    list match{
        case Nil=> true
        case head :: tail =>
            val tailOrdered: Boolean = foo(tail)
            tail match{
                case Nil => true
                case head2 :: tail2 => 
                head< head2 && tailOrdered
            }
    }

defined function foo

In [11]:
def foo(list:List[Int]): Boolean =
    list match{
        case head :: head2 :: tail =>
            val tailOrdered: Boolean = foo(head2 :: tail)
                head< head2 && tailOrdered
        case _ => true
    }

defined function foo

In [12]:
def foldRight(list: List[Int])(
    nil:Boolean,
    cons:(Int,Boolean)=>Boolean): Boolean = 
    list match{
        case Nil => nil
        case head::tail=> 
         val tailSol:Boolean = foldRight(tail)(nil,cons)
         cons(head,tailSol):Boolean
    }

defined function foldRight

In [13]:
//EL BUENO, USA RECURSIVIDAD Y GENERICIDAD
def foldRight[A,B](list: List[A])(
    nil:B,
    cons:(A,B)=>B): B = 
    list match{
        case Nil => nil
        case head::tail=> 
         val tailSol:B = foldRight(tail)(nil,cons)
         cons(head,tailSol):B
    }

defined function foldRight

In [17]:
def sorted(list:List[Int]): Boolean =
    foldRight(list)(
       (Nil, true): (List[Int],Boolean),
        (head:Int,tailSol: (List[Int],Boolean)) =>
            tailSol._1 match {
                case Nil => (head::tailSol._1,true)
                case head2::tail2 =>
                    (head :: tailSol._1,
                     head < head2 && tailSol._2)
            })._2

defined function sorted

In [15]:
sorted(List(1,2,3,4,5))

res14: Boolean = true

In [18]:
sorted(List(1,2,8,4,5))

res17: Boolean = false

Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

#### Improved type-inference

Technically, you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [ ]:

def multiply(list: List[Int]): Int = 
    foldRight(list)(1, (a, b) => a * b)
    //list.foldRight(1)(_*_)


In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [ ]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [ ]:
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [ ]:
def foldRight[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

In [ ]:
def foldRight[A, B, C](opt: Either[A, B])(left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (we will see catamorphisms later on). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. For instance:

In [20]:
def length[A](list: List[A]): Int = {
    var out: Int = 0
    for (e <- list)
        out += 1
    out
}

defined function length

In [19]:
def reverse[A](list: List[A]): List[A] = {
    var out: List[A] = Nil
    for (e <- list)
        out = e :: out
    out
} 

defined function reverse

The following implementation abstracts away the differences in the above functions, and declares two additional parameters: the initial value of a mutable variable, and the update function executed in every step of the loop.

In [23]:
def foldLeft[A,B](list: List[A])(initial: B,update :(B,A)=>B): B = {
    var out: B = initial
    for (e <- list)
        out = update(out:B, e:A)
    out
} 

defined function foldLeft

In [24]:
def reverse[A](list:List[A]):List[A]=
    foldLeft(list)(
        Nil: List[A],
        (out: List[A], e: A)=> 
            e :: out : List[A])

defined function reverse

In [25]:
def length[A](list:List[A]):Int=
    foldLeft(list)(
        0: Int,
        (out: Int, e: A)=> 
            out += 1 : Int)

defined function length

In [0]:
def length[A](list:List[A]):Int=
    list.foldLeft(0)
        ((out: Int, e: A)=>out+1)

cmd0.sc:3: not found: type A
        val res0_1 = ((out: Int, e: A)=>out+1)
                                    ^cmd0.sc:2: missing argument list for method foldLeft in trait LinearSeqOptimized
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `foldLeft _` or `foldLeft(_)(_)` instead of `foldLeft`.
    list.foldLeft(0)
                 ^Compilation Failed

: 

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
//@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

But we may also use the `foldRight` function to implement them:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [ ]:
foldRight[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [ ]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

This implementation works right with big lists:

In [ ]:
// foldRightTR[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [ ]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    list match {
        case Nil => out
        case head :: tail => 
            foldLeft(tail)(update(out, head))(update)
    }

This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [ ]:
def foldLeftAux[A, B](list: List[A])(update: (B, A) => B): B => B =
    ???

The original signature can then be implemented as follows: 

In [ ]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

## The `map` HOF

In [1]:
def lengths(list:List[String]):List[Int]=
    list match{
        case Nil=>Nil:List[Int]
        case head::tail=>
            val taillengths:List[Int]=lengths(tail)
            head.length::taillengths:List[Int]
    }

defined function lengths

In [2]:
def lengths(list:List[String]):List[Int]=
    list.foldRight(Nil:List[Int])((head,tailLengths)=>
        head.length :: tailLengths:List[Int])

defined function lengths

In [5]:
def evens(list:List[String]):List[Boolean]=
    map(list)(s => s.length % 2 == 0)

defined function evens

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../images/map.svg)

The implementation for lists can be done as follows:

In [6]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
     list match{
        case Nil=>Nil:List[B]
        case (head: A)::(tail: List[A])=>
            val tailMapped:List[B]=map(tail)(f)
            f(head)::tailMapped:List[B]
    }

defined function map

The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [3]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends FlatSpec with Matchers{
    
    val isEven: Int => Boolean = _ % 2 == 0
    
    "mapping an empty list" should "return an empty list" in {
        map(List())(isEven) shouldBe Nil
    }
    
    "mapping an non-empty list" should "only change its content" in {
        map(List(1))(isEven) shouldBe List(false)
        map(List(1,2))(isEven) shouldBe List(isEven(1),isEven(2))
        map(List(1,2,3))(isEven) shouldBe List(isEven(1),isEven(2),isEven(3))
    }
}

defined class TestMap

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [10]:
run(new TestMap(map[Int, Boolean]))

cmd8$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [4]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    list.foldRight(Nil:List[B])((head:A,tailMapped:List[B])=>f(head)::tailMapped:List[B])

defined function mapFR

In [5]:
run(new TestMap(mapFR))

cmd2$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [1]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends FlatSpec with Matchers{
    
    "mapping the `None` value" should "return `None`" in {
        map(None)(isEven) shouldBe None
    }
    
    "mapping a `Some` value" should "only change its content" in {
        map(Option(3))(isEven) shouldBe Some(false)
        map(Option(1))(isEven) shouldBe Some(false)
        map(Option(2))(isEven) shouldBe Some(true)
    }
    
        
}

cmd1.sc:6: not found: value isEven
        map(None)(isEven) shouldBe None
                  ^cmd1.sc:10: not found: value isEven
        map(Option(3))(isEven) shouldBe Some(false)
                       ^cmd1.sc:11: not found: value isEven
        map(Option(1))(isEven) shouldBe Some(false)
                       ^cmd1.sc:12: not found: value isEven
        map(Option(2))(isEven) shouldBe Some(true)
                       ^Compilation Failed

: 

The implementation is simple:

In [7]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    maybeA match {
        case None => None
        case Some(a) => Some(f(a))
    }

defined function mapOpt

In [ ]:
run(new TestMapOption(mapOpt))

## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../images/filter.svg)

In [14]:
def filterEven(list:List[Int]):List[Int]=
list.foldRight(Nil : List[Int])(
    (head:Int, tailFiltered:List[Int])=>
       if(head % 2 == 0 )head :: tailFiltered
       else tailFiltered : List[Int]

)

defined function filterEven

In [15]:
def filter[A](list:List[A])(predicate: A=>Boolean):List[A]=
list.foldRight(Nil : List[A])(
    (head:A, tailFiltered:List[A])=>
       if(predicate(head))head :: tailFiltered
       else tailFiltered : List[A]

)

defined function filter

In [16]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(List())(_ % 2 == 0) shouldBe List()
        filter(List(1))(_ % 2 == 0) shouldBe List()
        filter(List(1,3,5))(_ % 2 == 0) shouldBe List()
        filter(List(2,4,6))(_ % 2 == 0) shouldBe List(2,4,6)
    }
}

defined class TestFilterList

In [17]:
run(new TestFilterList(filter))

cmd15$Helper$TestFilterList:
filter
- should work


Using `foldRight` we can get a more modular implementation: 

### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [3]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(None)(_ % 2 == 0) shouldBe None
        filter(Some(1))(_ % 2 == 0) shouldBe None
        filter(Some(2))(_ % 2 == 0) shouldBe Some(2)
    }
}

defined class TestFilterOption

In [2]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    maybeA match {
        case None => None
        case Some(a) if predicate(a) => Some(a)
        case _ => None
    }

defined function filter

In [4]:
run(new TestFilterOption(filter))

cmd2$Helper$TestFilterOption:
filter
- should work


## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [9]:
class TestWords(
    words: List[String] => List[String]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in {
        words(paragraph1) shouldBe 
            List("En","un","lugar",
                 "de","la","Mancha",
                 "de","cuyo","nombre",
                 "no","quiero","acordarme")
    }
}

defined class TestWords

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [6]:
val paragraph: List[String] = List("en un lugar", "de la Mancha")
paragraph.map((phrase:String)=> phrase.split(" ").toList)
paragraph.map((phrase:String)=> phrase.split(" ").toList).flatten

paragraph: List[String] = List("en un lugar", "de la Mancha")
res5_1: List[List[String]] = List(
  List("en", "un", "lugar"),
  List("de", "la", "Mancha")
)
res5_2: List[String] = List("en", "un", "lugar", "de", "la", "Mancha")

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [26]:
def flatten[A](listOflists: List[List[A]]): List[A] = 
    listOflists.foldRight(Nil : List[A])(
        (head:List[A],tailConcatenated:List[A]) =>
            head ++ tailConcatenated : List[A]
    )

defined function flatten

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [30]:
def words(paragraph: List[String]): List[String] = 
    paragraph.map((phrase:String)=> phrase.split(" ").toList).flatten

defined function words

In [10]:
run(new TestWords(words))

cmd8$Helper$TestWords:
words
- should work


In [34]:
run(new TestWords(wordsFT))

cmd19$Helper$TestWords:
words
- should work


This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

In [35]:
def flatMap[A](list: List[A])(f: A => List[A]): List[A] = 
    list.map(f).flatten

defined function flatMap

![filter](../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [8]:
def words(paragraph: List[String]): List[String] = 
    paragraph.flatMap((phrase:String)=> phrase.split(" ").toList)

defined function words

### FlatMapping optional values

In [12]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    maybeA match {
        case None => 
            None
        case Some(a) => 
            cont(a)
    }

defined function flatMap

In [15]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends FlatSpec with Matchers {
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in {
        flatMap(None)(isPositiveEven) shouldBe None
        flatMap(Some(5))(isPositiveEven) shouldBe Some(false)
        flatMap(Some(-5))(isPositiveEven) shouldBe None
        flatMap(Some(0))(isPositiveEven) shouldBe Some(true)
        flatMap(Some(4))(isPositiveEven) shouldBe Some(true)
        
        
    }
}

defined class TestFlatMapOption

In [16]:
run(new TestFlatMapOption(flatMap))

cmd14$Helper$TestFlatMapOption:
flatMap
- should work


## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [17]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in {
        lengths(paragraph1) shouldBe 
            List(2, 2, 5, 2, 2, 6, 
                 2, 4, 6, 2, 6, 9)
    }
}

defined class TestLengths

We may try to implement this function imperatively using mutable variables:

In [18]:
def lengthsM(paragraph: List[String]): List[Int] = {
    var out: List[Int] = List()

    for (sentence <- paragraph){
        val words: List[String] =
            sentence.split(" ").toList
        for (word <- words)
            if (word != "") 
                out = word.length :: out
    }

    out.reverse
}


defined function lengthsM

and this works, of course: 

In [19]:
run(new TestLengths(lengthsM))

cmd16$Helper$TestLengths:
lengths
- should work


but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [20]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    paragraph.flatMap(sentence=>sentence.split(" ").toList)
    .filter(word => word !="": Boolean)
    .map(word=> word.length)


defined function lengthsHOF

In [21]:
run(new TestLengths(lengthsHOF))

cmd16$Helper$TestLengths:
lengths
- should work


This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [24]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    paragraph
    .flatMap(_.split(" ").toList)
    .filter(_ !="": Boolean)
    .map(_.length)

defined function lengthsFC

In [25]:
run(new TestLengths(lengthsFC))

cmd16$Helper$TestLengths:
lengths
- should work
